<a href="https://colab.research.google.com/github/doble196/NLP_Spacy/blob/main/nlp_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goal: Investigate the eBay challenge with the intention of intigrating Machine Learning and AI technics from class. Take advantage of the eBay datasets, learn, build, test, evaluate, and optimize our NLP product.

Plan:

Slicing both datasets for higher effeciency and accuracy

Start google colab for group collaboration

Keep learning and researching on NLP, SpaCy, and similar projects

Use Train Tagged Titles new dataset to build our pipline with custom NERs and patterns

Test the created pipline on a small set of Listing Title dataset

5.1 Loaded the original Training dataset for fuller entity list and better results

Get the Classification report of current model and identify where and what do we need to do to improve the model accuracy (ie. improve the pattern list)

Optimize the model

Optional: explore other libraries ie. sklearn nlp models

Presentation preparation and README file

In [1]:
%%capture
!pip install spacy.annotator

In [2]:
# Import all libraries and dependencies

import pandas as pd
import spacy
import numpy as np
import spacy_annotator as spa
from spacy.tokens import Span
from collections import Counter
from string import punctuation
from random import shuffle
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example
from spacy import displacy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import training dataset
from google.colab import files
uploaded = files.upload()
#read the file
#ttt = pd.read_csv('Train_Tagged_Titles.tsv', on_bad_lines = 'skip', sep = ',')
#ttt = ttt.replace(np.nan, 'Brand', regex=True)
#read the file
#lt_sm = pd.read_csv('Listing_Titles_sm.tsv')



Saving Train_Tagged_Titles.csv to Train_Tagged_Titles.csv
Saving Listing_Titles_sm.csv to Listing_Titles_sm.csv


In [5]:
ttt_df = pd.read_csv('Train_Tagged_Titles.csv',
    on_bad_lines = 'skip', sep = '\t')
ttt_df = ttt_df.replace(np.nan, 'Brand', regex=True)
#read the file
listing_titles_sm = pd.read_csv('Listing_Titles_sm.csv')

In [6]:
print(ttt_df.shape)
display(ttt_df)

(38209, 4)


,Record Number,Title,Token,Tag
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,Brand
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,Brand
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model
...,...,...,...,...
38204,5000,Botkier Sasha Medium Duffel Bag Coral Leather ...,Top,No Tag
38205,5000,Botkier Sasha Medium Duffel Bag Coral Leather ...,Closure,No Tag
38206,5000,Botkier Sasha Medium Duffel Bag Coral Leather ...,Retail,No Tag
38207,5000,Botkier Sasha Medium Duffel Bag Coral Leather ...,$,No Tag


In [7]:
listing_titles_sm.drop(columns = 'Unnamed: 0', inplace = True)
print(listing_titles_sm.shape)
display(listing_titles_sm)

(9985, 2)


,Record Number,Title
0,19989937,Authentic Louis Vuitton Monogram Bucket PM Sho...
1,19989938,Will Leather Canvas tote
2,19989939,Brand New with tags Ness Small Tweed bag
3,19989940,Dooney & Bourke Suede Leather Gia Satchel Hand...
4,19989941,Pirates of the Caribbean Canvas String Drawstr...
...,...,...
9980,19999917,' Wire Frame Octopus ' Tote Shopping Bag For L...
9981,19999918,Vintage Etienne Aigner Medium Oxblood Red Leat...
9982,19999919,Nine West Purse Shoulder Bag Handbag Camel Bro...
9983,19999920,New 2021 Ladies Tote Bag Women Crossbody Shoul...


## Strategies:

* Rules based approach follow https://youtu.be/wpyCzodvO3A 
* Train machine learning model to catch tokens that were spelled wrong https://www.youtube.com/watch?v=YBRF7tq1V-Q

* Custom NER training for specific use case


In [10]:
# Get all of the unique tags from ttt_df dataframe and change them to a list format
all_tags = ttt_df["Tag"].unique().tolist()

In [14]:
all_tags

['LOUIS',
 'VUITTON',
 'M40096',
 'Handbag',
 'Priscilla',
 'Multi-color',
 'canvas',
 'Petit',
 'Noe',
 'Drawstring',
 'Shoulder',
 'Bag',
 'Monogram',
 'Leather',
 'M42226',
 '39SD442',
 'Damier',
 'Azur',
 'Pochette',
 'Bosphore',
 'N51112',
 'LV',
 'Auth',
 'yt523',
 'GUCCI',
 'Bamboo',
 '2Way',
 'Brown',
 'fm1002',
 'Rank',
 'AB',
 'Vintage',
 'Gucci',
 'Sherry',
 'line',
 'PVC',
 'Clutch',
 'bag',
 'From',
 'Japan',
 'A128',
 '1970s',
 'NYC',
 'Bonnie',
 'Cashin',
 'Coach',
 'Gray',
 'Saddle',
 'Pouch',
 'Crossbody',
 'Epi',
 'Serviette',
 'Fermoir',
 'Business',
 'M54358',
 'gt2071',
 'Womens',
 'Chanel',
 'Leo',
 'Lion',
 'Flap',
 'Chevron',
 'Lambskin',
 'Medium',
 'White',
 'Authenticated',
 'Two',
 'Tone',
 'Women',
 'Voyageur',
 'Backpack',
 'Black',
 'Floral',
 '15.5',
 '.25',
 'TM-113',
 'Louis',
 'Vuitton',
 'Marley',
 'MM',
 'M94615',
 "'s",
 ',',
 'Fuchsia',
 'BF531772',
 'Josephine',
 'PM',
 'Blue',
 'Mini',
 'Lin',
 'Hand',
 '0J130140n',
 '16',
 'Canvas',
 'Great',
 

In [13]:
# Get all of the unique Tokens from ttt_df dataframe and change them to a list format
all_tokens = ttt_df["Token"].unique().tolist()

In [15]:
all_tokens

['LOUIS',
 'VUITTON',
 'M40096',
 'Handbag',
 'Priscilla',
 'Multi-color',
 'canvas',
 'Petit',
 'Noe',
 'Drawstring',
 'Shoulder',
 'Bag',
 'Monogram',
 'Leather',
 'M42226',
 '39SD442',
 'Damier',
 'Azur',
 'Pochette',
 'Bosphore',
 'N51112',
 'LV',
 'Auth',
 'yt523',
 'GUCCI',
 'Bamboo',
 '2Way',
 'Brown',
 'fm1002',
 'Rank',
 'AB',
 'Vintage',
 'Gucci',
 'Sherry',
 'line',
 'PVC',
 'Clutch',
 'bag',
 'From',
 'Japan',
 'A128',
 '1970s',
 'NYC',
 'Bonnie',
 'Cashin',
 'Coach',
 'Gray',
 'Saddle',
 'Pouch',
 'Crossbody',
 'Epi',
 'Serviette',
 'Fermoir',
 'Business',
 'M54358',
 'gt2071',
 'Womens',
 'Chanel',
 'Leo',
 'Lion',
 'Flap',
 'Chevron',
 'Lambskin',
 'Medium',
 'White',
 'Authenticated',
 'Two',
 'Tone',
 'Women',
 'Voyageur',
 'Backpack',
 'Black',
 'Floral',
 '15.5',
 '.25',
 'TM-113',
 'Louis',
 'Vuitton',
 'Marley',
 'MM',
 'M94615',
 "'s",
 ',',
 'Fuchsia',
 'BF531772',
 'Josephine',
 'PM',
 'Blue',
 'Mini',
 'Lin',
 'Hand',
 '0J130140n',
 '16',
 'Canvas',
 'Great',
 

In [16]:
# Create a blank English model
nlp = spacy.blank('en')
# Create the entity ruler and add to entity pipeline
ruler = nlp.add_pipe('entity_ruler')
# Create an empty list to collect all the patterns of the entity
patterns = []


In [17]:
# Get all of the tag names
for tag in all_tags:
  # Save list of Tokens under the tag name
  items = ttt_df["Token"].loc[ttt_df["Tag"] == f"{tag}"].tolist()
  # Loops through created list of Tokens
  for item in items:
    # Adds the new pattern to pattens list
    patterns.append({'label': f'{tag}', 'pattern': item})

In [18]:
print(patterns[:10])
print(patterns[-10:])

[{'label': 'Pattern', 'pattern': 'Monogram'}, {'label': 'Pattern', 'pattern': 'Chevron'}, {'label': 'Pattern', 'pattern': 'Floral'}, {'label': 'Pattern', 'pattern': 'Monogram'}, {'label': 'Pattern', 'pattern': 'Flower'}, {'label': 'Pattern', 'pattern': 'Owl'}, {'label': 'Pattern', 'pattern': 'GG'}, {'label': 'Pattern', 'pattern': 'FLORAL'}, {'label': 'Pattern', 'pattern': 'Patchwork'}, {'label': 'Pattern', 'pattern': 'Monogram'}]
[{'label': 'Character', 'pattern': 'Minnie'}, {'label': 'Character', 'pattern': 'Snoopy'}, {'label': 'Character', 'pattern': 'Harley'}, {'label': 'Character', 'pattern': 'HARRY'}, {'label': 'Character', 'pattern': 'Pompompurin'}, {'label': 'Character', 'pattern': 'Princess'}, {'label': 'Character', 'pattern': 'MICKEY'}, {'label': 'Character', 'pattern': 'Lilo'}, {'label': 'Character', 'pattern': 'Mickey'}, {'label': 'Character', 'pattern': 'Harry'}]


In [19]:
# add patterns list to model
ruler.add_patterns(patterns)

# Save model with patterns
nlp.to_disk('./ebay')

In [20]:
# Load model
nlp = spacy.load('./ebay')

In [21]:
# Create lists
tokens = ttt_df["Token"]
tags = ttt_df["Tag"]
combined_list = []

In [22]:
tokens

0            LOUIS
1          VUITTON
2           M40096
3          Handbag
4        Priscilla
           ...    
38204          Top
38205      Closure
38206       Retail
38207            $
38208          595
Name: Token, Length: 38209, dtype: object

In [23]:
tags

0         Brand
1         Brand
2           MPN
3          Type
4         Model
          ...  
38204    No Tag
38205    No Tag
38206    No Tag
38207    No Tag
38208    No Tag
Name: Tag, Length: 38209, dtype: object

In [ ]:
# Create list with all ttt_df tokens and tags grouped together
for i in range(len(tokens)):
  combined_list.append([tokens[i], tags[i]])

In [ ]:
combined_list

[['LOUIS', 'Brand'],
 ['VUITTON', 'Brand'],
 ['M40096', 'MPN'],
 ['Handbag', 'Type'],
 ['Priscilla', 'Model'],
 ['Multi-color', 'Color'],
 ['canvas', 'Fabric Type'],
 ['Multi-color', 'Color'],
 ['canvas', 'Fabric Type'],
 ['LOUIS', 'Brand'],
 ['VUITTON', 'Brand'],
 ['Petit', 'Model'],
 ['Noe', 'Brand'],
 ['Drawstring', 'Closure'],
 ['Shoulder', 'Type'],
 ['Bag', 'Type'],
 ['Monogram', 'Pattern'],
 ['Leather', 'Material'],
 ['M42226', 'MPN'],
 ['39SD442', 'No Tag'],
 ['LOUIS', 'Brand'],
 ['VUITTON', 'Brand'],
 ['Damier', 'Product Line'],
 ['Azur', 'Color'],
 ['Pochette', 'Product Line'],
 ['Bosphore', 'Model'],
 ['Shoulder', 'Type'],
 ['Bag', 'Type'],
 ['N51112', 'MPN'],
 ['LV', 'Brand'],
 ['Auth', 'No Tag'],
 ['yt523', 'No Tag'],
 ['GUCCI', 'Brand'],
 ['Bamboo', 'Product Line'],
 ['2Way', 'Features'],
 ['Shoulder', 'Type'],
 ['Bag', 'Type'],
 ['Leather', 'Material'],
 ['Brown', 'Color'],
 ['Auth', 'No Tag'],
 ['fm1002', 'MPN'],
 ['Rank', 'No Tag'],
 ['AB', 'No Tag'],
 ['Vintage', 'No T

In [24]:
# Collecting testing dataset and convert testing data to list format
sentence_list = listing_titles_sm['Title'].tolist()

In [25]:
sentence_list

['Authentic Louis Vuitton Monogram Bucket PM Shoulder Bag M42238 LV 54525',
 'Will Leather Canvas tote',
 'Brand New with tags Ness Small Tweed bag',
 'Dooney & Bourke Suede Leather Gia Satchel Handbag CHESTNUT,Pre-owned',
 'Pirates of the Caribbean Canvas String Drawstring Backpack Gym Bag p27 w0066',
 'authentic gucci marmont handbag',
 'Coach Large Tote in Signature Canvas , Black',
 'Lovely Beaded Handbag Purse , Lime Green Silk , Preston & York',
 'Auth LOUIS VUITTON Pochette Double Zip M69203 Monogram Reverse Shoulder Bag',
 'RingStix Lite Ring Toss Game by World Market',
 'montana west crossbody / wristlet',
 'KAVU Outdoor Shoulder Bag Tote Carryall Black & Cream Canvas',
 'Ladies Guess black handbag - brand new with tags',
 'Ahilya Travel Bag',
 'r2059 Auth CHANEL Black Quilted Calfskin CC Tag Boston Travel Shoulder Hand Bag',
 'Mr Graby • Number One Teacher • Tote Bag • School Leaving Gift',
 'Marylai Women Black Leather Backpack One Size',
 'Prada Womens Tessuto Logo Zip Top 

In [26]:
# Load testing data list to saved ebay nlp model to extract entity text and its label
dta = []
label =[]
for i in sentence_list:
  doc = nlp(i)
  for ent in doc.ents:
      text =[[ent.text,ent.label_]]
      # label = [[ent.label_]]
      for i in text:
        dta.append(i)
      # # for i in label:
      # #   label.append(i)

In [27]:
dta

[['Louis', 'Brand'],
 ['Vuitton', 'Brand'],
 ['Monogram', 'Brand'],
 ['PM', 'Brand'],
 ['Shoulder', 'Brand'],
 ['Bag', 'Brand'],
 ['LV', 'Brand'],
 ['Leather', 'Material'],
 ['Canvas', 'Brand'],
 ['tote', 'Brand'],
 ['Brand', 'Brand'],
 ['New', 'Model'],
 ['Small', 'Size'],
 ['bag', 'Brand'],
 ['Dooney', 'Brand'],
 ['&', 'Brand'],
 ['Bourke', 'Brand'],
 ['Suede', 'Brand'],
 ['Leather', 'Brand'],
 ['Satchel', 'Brand'],
 ['-', 'Brand'],
 ['of', 'Brand'],
 ['the', 'Model'],
 ['Canvas', 'Brand'],
 ['Drawstring', 'Closure'],
 ['Backpack', 'Brand'],
 ['Gym', 'Brand'],
 ['Bag', 'Brand'],
 ['gucci', 'Brand'],
 ['Coach', 'Brand'],
 ['Large', 'Size'],
 ['Tote', 'Brand'],
 ['in', 'Brand'],
 ['Signature', 'Pattern'],
 ['Canvas', 'Brand'],
 ['Black', 'Color'],
 ['Beaded', 'Accents'],
 ['Purse', 'Brand'],
 ['Lime', 'Color'],
 ['Green', 'Color'],
 ['Silk', 'Material'],
 ['&', 'Brand'],
 ['York', 'Brand'],
 ['LOUIS', 'Brand'],
 ['VUITTON', 'Brand'],
 ['Pochette', 'Model'],
 ['Double', 'Pattern'],
 ['Z

In [28]:
# Create a function to get ebay nlp model performance evaluation metrix
def my_evaluate(model, examples):
  # Create Scorer class
  scorer = Scorer() 
  # Create empty list
  example = []
  # Loop through examples
  for input_, annotations in examples: 
    print(input_)
score = my_evaluate(nlp,dta)

Streaming output truncated to the last 5000 lines.
Convertible
Crossbody
Backpack
Purse
Yellow
Black
Quilted
Strap
Body
Shoulder
Bag
Orange
Women
's
Purse
Cream
Leather
London
Grey
Floral
Pink
Satin
Crystal
Oval
Clutch
Bag
MCM
Medium
Tote
Bag
BLUE
Crossbody
Fendi
Black
Leather
Tote
Bag
Michael
Kors
Manhattan
Metallic
Medium
Top
Handle
School
Satchel
Black
Ferragamo
Louis
vuitton
Hand
Bag
Monogram
-
Rebecca
Minkoff
Women
's
Red
Orange
Leather
Bag
NINE
WEST
SET
BEIGE
TAN
CANVAS
LEATHER
MEDIUM
Clutch
Bag
Black
Mini
Tulip
Black
Leather
Bag
Out
Le
Black
Leather
Hand
Navy
Blue
Leather
Crossbody
Bag
Bag
GREY
Women
's
Clutch
Bag
Michael
Kors
Small
Bag
Red
New
Ladies
Style
Army
Black
Bag
Girls
tote
bag
New
Cognac
Leather
Women
's
Backpack
Black
Signature
C
Coach
Gucci
Shoulder
Bag
One
Razor
Canvas
Vera
Bradley
Backpack
NEW
A
NEW
Brent
&
Dark
Brown
Leather
Coach
Derby
Tote
Bright
Poppy
Red
NEW
Blue
hand
Bag
Long
Strap
Coach
19
Rose
Black
Leather
michael
kors
Light
Blush
Clutch
Shoulder
COACH
All

In [30]:
# Create a function to get ebay nlp model performance evaluation metrix
def my_evaluate(model, examples):
  # Create Scorer class
  scorer = Scorer() 
  # Create empty list
  example = []
  # Loop through examples
  for input_, annotations in examples: 
    # Predict
    pred = model(input_)
    print(pred, annotations)
    # Create Example instance
    temp = Example.from_dict(pred, dict.fromkeys(annotations))
    # append to example list
    example.append(temp)
  # Get Scores
  scores = scorer.score(example)
  return scores


In [29]:
# Get score of model from my_evaluate
score = my_evaluate(nlp,dta)
print(score)

Streaming output truncated to the last 5000 lines.
Crossbody
Backpack
Purse
Yellow
Black
Quilted
Strap
Body
Shoulder
Bag
Orange
Women
's
Purse
Cream
Leather
London
Grey
Floral
Pink
Satin
Crystal
Oval
Clutch
Bag
MCM
Medium
Tote
Bag
BLUE
Crossbody
Fendi
Black
Leather
Tote
Bag
Michael
Kors
Manhattan
Metallic
Medium
Top
Handle
School
Satchel
Black
Ferragamo
Louis
vuitton
Hand
Bag
Monogram
-
Rebecca
Minkoff
Women
's
Red
Orange
Leather
Bag
NINE
WEST
SET
BEIGE
TAN
CANVAS
LEATHER
MEDIUM
Clutch
Bag
Black
Mini
Tulip
Black
Leather
Bag
Out
Le
Black
Leather
Hand
Navy
Blue
Leather
Crossbody
Bag
Bag
GREY
Women
's
Clutch
Bag
Michael
Kors
Small
Bag
Red
New
Ladies
Style
Army
Black
Bag
Girls
tote
bag
New
Cognac
Leather
Women
's
Backpack
Black
Signature
C
Coach
Gucci
Shoulder
Bag
One
Razor
Canvas
Vera
Bradley
Backpack
NEW
A
NEW
Brent
&
Dark
Brown
Leather
Coach
Derby
Tote
Bright
Poppy
Red
NEW
Blue
hand
Bag
Long
Strap
Coach
19
Rose
Black
Leather
michael
kors
Light
Blush
Clutch
Shoulder
COACH
All
Over
Beige


In [ ]:
print(nlp.pipe_names)